# 第6章 迭代

Python从可迭代的对象(`__next__`)中获取迭代器（`__iter__`）。

## 6.1 自定义迭代器



In [1]:
import re
import reprlib

RE_WORD = re.compile('\w+')


class Sentence:
     def __init__(self, text):
         self.text = text
         self.words = RE_WORD.findall(text)
            
     def __repr__(self):
     	return 'Sentence(%s)' % reprlib.repr(self.text)
    
     def __iter__(self):
         # 迭代self.words
         for word in self.words:
            # 产出当前的 word
         	yield word 
            
         # 这个return可以不写，不论写不写都不会报StopIteration异常   
         return 
        
# 完成！ 不需要定义可迭代的对象

## 6.2 itertools

Python自带的,和迭代器相关的`itertools`标准库模块：

### 6.2.1 itertools.producr()

对于多层嵌套函数，可以用`producr` 扁平化：




In [2]:
from itertools import product


for i in range(1,3):
    for j in range(1,3):
        print(f"{i}*{j}={i*j}")
print("--------")
        
for i,j in product(range(1,3), range(1,3)):
    print(f"{i}*{j}={i*j}")

1*1=1
1*2=2
2*1=2
2*2=4
--------
1*1=1
1*2=2
2*1=2
2*2=4


### 6.2.2 迭代边界条件判定

有时候，我们执行迭代时候，会对迭代的当前数据进行判定，此时我们可以使用：`itertools.takewhile(条件函数, 生成器)` 优化：

In [14]:
from itertools import takewhile

def iter_foo():
    for i,j in product(range(1,3), range(1,3)):
        yield i*j

def is_not_one(index):
    if index > 1:
        return False
    else:
        return True

for item in takewhile(is_not_one, iter_foo()):
    print(item)

1


### 6.2.3 其他函数

`itertools.count()`:如果不写参数，会从0开始生成整数，提供可选的`start`和`step` 参数，`list(itertools.count())` 电脑直接爆炸。

`itertools.cycle()`: 会从指定序列中反复不断地生成元素。


In [29]:
import itertools
import sys
import time

write, flush = sys.stdout.write, sys.stdout.flush

def loop_foo():
    for i in  itertools.cycle('|/-\\'):
        yield i

def exit_foo():
    index = 0
    def min_exit_foo(value):
        nonlocal index
        index += 1
        if index > 20 : 
            return False 
        else:
            return True
    return min_exit_foo
    
is_exit = exit_foo()    
for char in takewhile(is_exit,itertools.cycle('|/-\\')):
    write(char)
    flush()
    # 实现动画地原因，使用退格符把光标移回来
    write('\x08' * len(char)) 
    time.sleep(.1)

## 6.3 break

多层嵌套的循环，一个break，并不能跳出所有。所以可以把嵌套循环单独拿出来，当作一个函数，然后使用`return`而不是`break`.

In [34]:
def get_24():
    for i in range(1,20):
        for j in range(1,20):
            if i*j == 24:
                return i,j 
get_24()

(2, 12)

# 第7章 函数

## 7.1 函数输入参数

可变参数和不定参数，让Python自由度极大的上升。函数接收的参数最好不要超过3个。通过加入`*`，让后面的参数全部变为关键参数，可让代码更加健壮。
> Python3.8 加入了新的特性支持“仅限位置参数”，但是没啥实用性。

In [50]:
def foo(name, gender, *, fit):
    print(name, gender, fit)

foo('rohan', 'male' )

TypeError: foo() missing 1 required keyword-only argument: 'fit'

## 7.2 函数返回值

### 7.2.1尽量返回一种类型

Python轻易就可以做到静态语言做不到的事：函数支持返回不同类型的结果。但是不要这么做。这种多功能函数，看起来“多才多艺”。其实会增加复杂度，增加维护成本。

更好的做法是，一个功能拆分一个函数。

### 7.2.2 警惕None
函数返回None的场景：
- 操作类函数或无返回值函数的默认值
- 尝试“搜索”、“查询”函数，意料之中的结果

除此以外的场景，你都应该用抛出异常更加合理。

## 7.3 内置模块functools

`partial`偏函数：可以让有同样参数的函数，复用时更加扁平。更加突出区别：

In [49]:
from functools import partial


def foo(gender, address, name, email, age):
    print(gender, address, name, email, age)
    
foo_male = partial(foo, gender = 'male', address = 'China')
foo_male(name='Rohan', email='rohan@gmail.com', age='18')

male China Rohan rohan@gmail.com 18


`lru_catche(max_size=None)`给函数增加缓存功能：如果某个函数的返回值相对于输入值是固定的，那么可以使用缓存功能，避免重复计算,`max_size`表示最多保存多少缓存结果。默认为128。

In [8]:
import timeit
from functools import lru_cache

@lru_cache()
def fibonacci(n):
    if n < 2 :
        return n
    return fibonacci(n-2) + fibonacci(n-1)

def none_catch_fibonacci(n):
    if n < 2 :
        return n
    return fibonacci(n-2) + fibonacci(n-1)


time = timeit.timeit(setup='from __main__ import fibonacci',stmt = 'fibonacci(20)')
none_catch_time = timeit.timeit(setup='from __main__ import none_catch_fibonacci',stmt = 'none_catch_fibonacci(20)')
print(f'time: {time}; none_catch_time: {none_catch_time}')

time: 0.06258358400009456; none_catch_time: 0.12867616699986684


## 7.4 函数抽象

一个单独的功能，写成一个单一的函数，这没问题。在讲多个函数组合成一个大的层的时候。也要注意业务逻辑的抽象。

保证一个函数内抽象级别一致，函数的职责更简单，代码更易读。

## 7.5 递归的局限性

在其他语言中，为了避免递归调用栈过深，使用尾调用优化，讲递归优化成循环，以避免嵌套层级过深。一定要低于`sys.getrecursionlimit()`的深度。

如果可以，尽量将递归改写成循环。

In [9]:
import sys

sys.getrecursionlimit()

3000